# Data Cleaning

In [126]:
import numpy as np
import pandas as pd
import re
from functools import reduce

In [127]:
df = pd.read_csv("../data/original.csv")
df.head()

,Unnamed: 0,Pos,Name,Club,Cat,Swim,T1,Bike,T2,Run,Time
0,1,1,FRODENO Jan (2),Laz Saarbruecken,MPRO - 1,00:22:501,00:01:2710,02:02:011,00:01:5221,01:11:252,03:39:35
1,2,2,CLAVEL Maurice (24),NaN,MPRO - 2,00:23:3311,00:01:3015,02:04:543,00:01:323,01:12:144,03:43:43
2,3,3,TAAGHOLT Miki Morck (21),Ttsdu,MPRO - 3,00:22:574,00:01:141,02:05:526,00:02:0148,01:14:238,03:46:27
3,4,4,STRATMANN Jan (13),Triathlon Team Witten,MPRO - 4,00:22:502,00:01:3118,02:08:208,00:01:5633,01:13:336,03:48:10
4,5,5,MOLINARI Giulio (22),C. S. Carabinieri,MPRO - 5,00:22:585,00:01:4968,02:05:114,00:02:0768,01:17:2117,03:49:26


### Remove Uninteresting Columns

We initially remove the unnamed column, the athlete name, and the club.

In [128]:
cols = df.columns
df.drop(columns=[cols[0], 'Name', 'Club'], inplace=True)
df.head()

,Pos,Cat,Swim,T1,Bike,T2,Run,Time
0,1,MPRO - 1,00:22:501,00:01:2710,02:02:011,00:01:5221,01:11:252,03:39:35
1,2,MPRO - 2,00:23:3311,00:01:3015,02:04:543,00:01:323,01:12:144,03:43:43
2,3,MPRO - 3,00:22:574,00:01:141,02:05:526,00:02:0148,01:14:238,03:46:27
3,4,MPRO - 4,00:22:502,00:01:3118,02:08:208,00:01:5633,01:13:336,03:48:10
4,5,MPRO - 5,00:22:585,00:01:4968,02:05:114,00:02:0768,01:17:2117,03:49:26


In [129]:
# Helper functions for processing/filtering

def all(ls: list) -> bool:
    return reduce(lambda a, b: a and b, ls)

def startswith(s: str, prefix: str) -> bool:
    s_ = str(s)
    return (len(s_) >= len(prefix)) and all(map(lambda tup: tup[0] == tup[1], zip(s_,prefix)))

### Convert 'Cat' column into Category Code

In [130]:
# Remove the Category placing suffix

def parsecategory(s: str) -> str:
    s_ = str(s)
    res = re.split(r"\W+", s_)
    return res[0] if len(res) > 0 else ""

df['Cat'] = df['Cat'].apply(parsecategory)

In [131]:
# Remove 'nan' rows

df = df.loc[df['Cat'].apply(lambda s: s != 'nan')]

In [132]:
# List all the distinct categories

cats = np.unique(df['Cat'])

In [133]:
# Map each category to an integer encoding

def indexof(v, arr) -> int:
    for i, x in enumerate(arr):
        if v == x:
            return i
    return -1

def catmap(v: str) -> int:
    return indexof(v, cats)

df['Cat'] = df['Cat'].apply(catmap)
df.head()

,Pos,Cat,Swim,T1,Bike,T2,Run,Time
0,1,17,00:22:501,00:01:2710,02:02:011,00:01:5221,01:11:252,03:39:35
1,2,17,00:23:3311,00:01:3015,02:04:543,00:01:323,01:12:144,03:43:43
2,3,17,00:22:574,00:01:141,02:05:526,00:02:0148,01:14:238,03:46:27
3,4,17,00:22:502,00:01:3118,02:08:208,00:01:5633,01:13:336,03:48:10
4,5,17,00:22:585,00:01:4968,02:05:114,00:02:0768,01:17:2117,03:49:26


### Parse the time fields into seconds

In [134]:
def parsetime(s: str) -> float:
    res = re.split(':', s)
    if len(res) != 3:
        return 0
    else:
        hrs, mins, secs = res
        secs = secs[0:2]
        return 60*60*int(hrs) + 60*int(mins) + int(secs)


In [135]:
timecols = df.columns[2:]
timecols

for timecol in timecols:
    df[timecol] = df[timecol].apply(parsetime)

In [136]:
# remove any rows with values of 0

df = df.loc[df[timecols].apply(lambda row: all(map(lambda col: col != 0, [row[col] for col in timecols])), axis=1)]
df

,Pos,Cat,Swim,T1,Bike,T2,Run,Time
0,1,17,1370,87,7321,112,4285,13175
1,2,17,1413,90,7494,92,4334,13423
2,3,17,1377,74,7552,121,4463,13587
3,4,17,1370,91,7700,116,4413,13690
4,5,17,1378,109,7511,127,4641,13766
...,...,...,...,...,...,...,...,...
795,796,11,2345,213,9887,208,6624,19277
796,797,13,2201,200,9913,237,6727,19278
797,798,11,2481,338,9599,163,6698,19279
798,799,12,2436,128,9676,168,6878,19286


### Output Clean Data to CSV

In [137]:
df.to_csv('../data/cleaned.csv')